# ソルバーの実行

## 準備

In [8]:
import gurobi as gp
import datetime
import random
import numpy as np
random.seed(1)
import os

In [9]:
def group(i,l): # アイテムiがグループGに属しているかどうかをバイナリで返す関数
    for j in l:
        for k in j:
            if k==i:
                return 1
            else:
                return 0

In [10]:
def group(i,l): # アイテムiがグループGに属しているかどうかをバイナリで返す関数
    if i in l:
        return 1
    else:
        return 0

In [20]:
def solve(moel_name,n,w,C,E,M,r,bin_cost):
    model = gp.Model(name = model_name)
    #model.setParam('TimeLimit', 10000) #制限時間
    #解
    ans_x = {}
    ans_y = {}
    ans_b = {}
    #定数定義
    m = len(M)
    z = {}
    #変数定義
    x = {}
    y = {}
    b = {}
    for l in range(m):
        for k in range(n):
            y[k] = model.addVar(lb=0, ub=1, vtype='B', name='y[%s]'%k)
            b[k,l] = model.addVar(lb=0, ub=1, vtype='B', name='b[%s,%s]'%(k,l))
            for i in range(n):
                x[i,k] = model.addVar(lb=0, ub=1, vtype='B', name='x[%s,%s]'%(i,k))
                z[i,l] = group(i,M[l])
                #if l==0 and z[i,l]==1:
                    #print(i)
                
    #print(z)
    model.update()
        
    #目的関数
    ##model.setObjective(gp.quicksum(bin_cost*y[k] for k in range(n)) + gp.quicksum(r[l]*gp.quicksum(b[k,l] for k in range(n)) for l in range(m)) - sum(r), sense=gp.GRB.MINIMIZE)
    model.setObjective(gp.quicksum(y[k] for k in range(n)) + gp.quicksum(gp.quicksum(r[l]*b[k,l] for l in range(m)) for k in range(n)) - sum(r), sense=gp.GRB.MINIMIZE)

    #制約式
    for k in range(n):
        model.addConstr(gp.quicksum(w[i]*x[i,k] for i in range(n))<=C*y[k], 'Binlimit[%s]'%k) #ビンの容量を超えない
    for i in range(n):
        model.addConstr(gp.quicksum(x[i,k] for k in range(n))==1, 'Assign[%s]'%i) #すべてのアイテムはビンに詰め込まれている
        for k in range(n):
            model.addConstr(x[i,k]<=y[k], 'Strong[%s,%s]'%(i,k)) #強い定式化
    for e in E:
        for k in range(n):
            model.addConstr(x[e[0],k]+x[e[1],k]<=1,'Conflict[%s,%s]'%(e[0],e[1])) #コンフリクト
    for k in range(n):
        for l in range(m):
            for j in range(n):
                model.addConstr(b[k,l]>=x[j,k]*z[j,l],'Group[%s]'%l) #同じグループに属するアイテムは同じビンに割り当てられる
    
    #解の出力
    ##print("↓点線の間に、Gurobi Optimizerからログが出力")
    ##print("-" * 100)

    model.optimize()
    #model.write(f"{model_name}.rlp")
    
    ##print("-" * 100)
    ##print()
    
    if model.Status == gp.GRB.OPTIMAL:
        for v in model.getVars():
            if "x" in v.varName:
                #print("%s = %g"%(v.varName,v.x))
                ans_x[v.varName] = int(v.x)
            if "y" in v.varName:
                ans_y[v.varName] = int(v.x)
            if "b" in v.varName:
                ans_b[v.varName] = int(v.x)

        #print('Opt Value = ', model.ObjVal)
        print("x",ans_x)
        print("bin\t",sum(ans_y.values()))
        print("penalty\t",sum(ans_b.values())*r[0])
        print("Objective value\t",sum(ans_y.values())+(sum(ans_b.values()))*r[0]-sum(r))

        return model.ObjVal

In [21]:
#入力, 出力を書き出す階層を指定
path = os.path.abspath(".")
path

'/Users/yutaokamoto/Desktop/yutaokamoto_mac/Seminar/benchmark'

In [22]:
for instance_name in ["w120","w250","w500","w1000"]:
    for instance_number in range(10):
        for density in range(10):
            
            if (instance_name=="w120") and (instance_number==0) and (density==0):################################
                
                n = 0
                V = []
                C = 150
                w = []
                E = []
                M = []
                group_split_cost = 1
                r = []
                bin_cost = 5
                ans_x = []
                ans_y = []
                
                file_input = path + "/" + "input" + "/" + str("i"+str(instance_name[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + ".txt" #インスタンスの規模_インスタンスの番号_グラフの密度
                with open(file_input,mode="r") as f:
                    for line in f:
                        if "Number of items\t" in line:
                            n = int(line.split("\t")[-2])
                            V = list(range(n))
                        elif "Capacity of bins\t" in line:
                            C = int(line.split("\t")[-2])
                        elif "w\t" in line:
                            w.append(int(line.split("\t")[-1]))
                        elif "e\t" in line:
                            E.append(tuple(map(int,line.split("\t")[-1].rstrip("\n").strip("(").strip(")").split(","))))
                        elif "g\t" in line:
                            M.append(tuple(map(int,line.split("\t")[-1].rstrip("\n").strip("(").strip(")").strip(",").split(","))))
                        elif "Costs of spliting groups\t" in line:
                            r = [int(i) for i in line.split("\t")[-2][1:-1].split(",")]
                        elif "Costs of using bins\t" in line:
                            bin_cost = int(line.split("\t")[-2])
                            
                #print(M[0])
                
                #コンフリクト
                conf={}
                n_conf = len(E)
                for i in range(n_conf):
                    temp = [0]*n
                    temp[E[i][0]-1] = 1
                    temp[E[i][1]-1] = 1
                    conf[f"conf{i}"] = temp

                file_output =  path + "/" + "output" + "/" + "Gurobi" + "/" + str("i"+str(instance_name[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + "_" + "10000+s" + ".txt"
                model_name = str(file_output)

                objval = 0
                r = list(map(lambda x:x/bin_cost,r))
                bin_cost = bin_cost/bin_cost
                objval = solve(model_name,n,w,C,E,M,r,bin_cost)
                
                with open(file_output,mode="w") as f:
                    f.write("Object value\t"+str(objval)+"\n\n")

Optimize a model with 129840 rows, 117120 columns and 187320 nonzeros
Variable types: 0 continuous, 117120 integer (117120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 105.0000000
Presolve removed 107400 rows and 101880 columns
Presolve time: 0.63s
Presolved: 22440 rows, 15240 columns, 117360 nonzeros
Variable types: 0 continuous, 15240 integer (15240 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.69s

Solved with primal simplex

Root relaxation: objective 4.945333e+01, 14461 iterations, 2.22 seconds
Total elapsed time = 6.55s
Total elapsed time = 14.29s
Total elapsed time = 17.92s
Total elapsed time = 22.74s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Ti

  1057   649   51.15665   73  309   54.20000   50.31069  7.18%  1325  715s
  1059   651   51.16023   80  471   54.20000   50.31069  7.18%  1322  721s
  1060   651   53.80000   54  592   54.20000   50.31069  7.18%  1321  728s
  1061   652   54.00000   90  578   54.20000   50.31069  7.18%  1320  731s
  1062   653   50.77020   57  564   54.20000   50.34478  7.11%  1318  740s
  1064   654   51.45781  113  584   54.20000   50.37659  7.05%  1316  751s
  1067   656   50.55600   21  510   54.20000   50.39847  7.01%  1312  755s
  1072   659   50.54000   32  249   54.20000   50.40000  7.01%  1306  764s
  1073   660   51.28576   79  349   54.20000   50.40000  7.01%  1305  766s
  1074   661   50.48617   18  267   54.20000   50.40000  7.01%  1304  775s
  1076   662   50.48871   42  248   54.20000   50.40000  7.01%  1301  794s
  1077   666   50.80000   13  282   54.20000   50.40000  7.01%  1459  811s
  1079   669   50.80000   14  285   54.20000   50.40000  7.01%  1461  820s
  1083   672   50.80000  

 12187  7506   51.06428   62  380   54.00000   50.52819  6.43%  1526 3172s
 12598  7743   53.50218  129  335   54.00000   50.53139  6.42%  1541 3245s
 13077  8049   50.53144   35  431   54.00000   50.53143  6.42%  1545 3318s
 13501  8358   52.70885   52  402   54.00000   50.53143  6.42%  1554 3392s
 13878  8506   50.86413   54  564   54.00000   50.53143  6.42%  1576 3473s
 14199  8737   53.69030  186  328   54.00000   50.53143  6.42%  1561 3550s
 14766  8988   50.57203   42  417   54.00000   50.53228  6.42%  1566 3621s
 15146  9208   50.54651   38  430   54.00000   50.53307  6.42%  1575 3701s
 15343  9306   51.82900   49  372   54.00000   50.53388  6.42%  1585 3776s
 15722  9547   51.93827   38  379   54.00000   50.53388  6.42%  1594 3944s
 15786  9597   52.68952   48  352   54.00000   50.53422  6.42%  1595 4016s
 16189  9838   52.71788  108  397   54.00000   50.53422  6.42%  1596 4092s
 16681 10228     cutoff   63        54.00000   50.53422  6.42%  1585 4175s
 16776 10282   51.69359  

 28438 15905   53.40267  134  335   54.00000   50.55102  6.39%  1768 10445s
 28791 16022   51.03819   49  528   54.00000   50.55102  6.39%  1776 10543s
 29007 16073   50.74034   50  611   54.00000   50.55102  6.39%  1797 10647s
 29208 16138   50.55102   52  371   54.00000   50.55102  6.39%  1814 10753s
 29570 16306   51.12564   66  516   54.00000   50.55102  6.39%  1815 10856s
 29988 16462   50.55102   58  509   54.00000   50.55102  6.39%  1824 10977s
 30280 16559   50.55175   53  534   54.00000   50.55102  6.39%  1826 11096s
 30762 16763   50.83116   60  475   54.00000   50.55102  6.39%  1832 11209s
 31103 16938   50.67238   56  581   54.00000   50.55102  6.39%  1834 11330s
 31443 17050   52.08077   96  442   54.00000   50.55102  6.39%  1840 11533s
 31535 17058   52.19856  119  436   54.00000   50.55102  6.39%  1845 11667s
 31957 17209   52.84061   80  477   54.00000   50.55102  6.39%  1853 11802s
 32144 17213   52.15988   60  559   54.00000   50.55102  6.39%  1872 11931s
 32579 17394

 67960 31729   51.85102   63  529   54.00000   50.60581  6.29%  2640 40077s
 68275 31937   50.77250   49  519   54.00000   50.60999  6.28%  2648 40480s
 68498 32030     cutoff   72        54.00000   50.61185  6.27%  2658 40868s
 69381 32726   51.07366   68  627   54.00000   50.61481  6.27%  2644 41230s
 69760 33011   50.76787   59  459   54.00000   50.61541  6.27%  2647 41692s
 70006 33141   50.99840   55  619   54.00000   50.61541  6.27%  2656 42207s
 70283 33296   51.00201   60  584   54.00000   50.61763  6.26%  2665 42622s
 70553 33419   51.49514   80  509   54.00000   50.61774  6.26%  2676 43047s
 70907 33625     cutoff   73        54.00000   50.61774  6.26%  2682 43358s
 71389 33954   52.22409   93  644   54.00000   50.61953  6.26%  2682 43757s
 71672 34149   51.58050   75  544   54.00000   50.61953  6.26%  2692 44198s
 72032 34406   52.69042  138  409   54.00000   50.61997  6.26%  2701 44720s
 72361 34604   52.26477   73  462   54.00000   50.61997  6.26%  2708 45149s
 72776 34816

# 解の図示 

## ビンごとに辞書に登録 

In [21]:
node_dict = {}
for Item,Bin in ans_x:
    if ans_x[Item,Bin] == 1:
        Item = int(Item)
        Bin = int(Bin)
        try:
            node_dict[Bin] = node_dict[Bin] + [Item]
        except:
            node_dict[Bin] = [Item]
        
node_dict

{0: [0]}

In [16]:
node_dict = {}
for Item,Bin in ans_x:
    if ans_x[Item,Bin] == 1:
        Item = int(Item)
        Bin = int(Bin)
        node_dict.setdefault(Bin,[])
        node_dict[Bin] = node_dict[Bin] + [Item]
    
node_dict

{0: [0]}

In [17]:
node_dict = {}
for Item,Bin in ans_x:
    if ans_x[Item,Bin] == 1:
        Item = int(Item)
        Bin = int(Bin)
        node_dict[Bin] = node_dict.setdefault(Bin,[]) + [Item]
    
node_dict

{0: [0]}

## 描画

In [18]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import random
G = nx.Graph()

for v in range(n): # グラフにノードを追加
    G.add_node(v)
    
pos = nx.spring_layout(G, k=1)
plt.figure(figsize=(10,10))


for l in node_dict.values():
    node_color = random.sample(matplotlib.colors.cnames.keys(),1)
    nx.draw_networkx_nodes(G, pos, nodelist=[i for i in l], node_color=node_color, alpha=0.6, node_size=4000)
    nx.draw_networkx_labels(G, pos, font_size=50, font_family="Yu Gothic", font_weight="bold")
    
%matplotlib inline
plt.show()
#plt.savefig("figure.png")

In [19]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import random
G = nx.Graph()

for v in range(n): # グラフにノードを追加
    G.add_node(v)
    
plt.figure(figsize=(10,10))

for b in range(n):
    node_color = random.sample(matplotlib.colors.cnames.keys(),1)
    for i in range(n):
        if ans_x[str(i),str(b)]==1:
            nx.draw_networkx_nodes(G, pos, nodelist=[i],node_color=node_color, alpha=0.6, node_size=4000)
            nx.draw_networkx_labels(G, pos, font_size=50, font_family="Yu Gothic", font_weight="bold")

plt.show()
#plt.savefig("figure.png")

KeyError: ('10', '0')

findfont: Font family ['Yu Gothic'] not found. Falling back to DejaVu Sans.


In [ ]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import random
G = nx.Graph()

for v in range(n): # グラフにノードを追加
    G.add_node(v)
    
plt.figure(figsize=(10,10))

for b in map(lambda x:x[1],ans_x.keys()):
    node_color = random.sample(matplotlib.colors.cnames.keys(),1)
    for i in list(set(map(lambda x:x[0],ans_x.keys()))):
        if ans_x[str(i),str(b)]==1:
            nx.draw_networkx_nodes(G, pos, nodelist=[int(i)],node_color=node_color, alpha=0.6, node_size=4000)
            nx.draw_networkx_labels(G, pos, font_size=50, font_family="Yu Gothic", font_weight="bold")

plt.show()
#plt.savefig("figure.png")